In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
import scipy.stats as stats
from scipy.stats import gamma
from scipy.stats import ortho_group

import cupy as cp
import cupyx.scipy 

from Test.hsic_naive import IndpTest_naive
from Test.hsic_lkgau import IndpTest_LKGaussian
from Test.hsic_lklap import IndpTest_LKLaplace
from Test.hsic_lkwgau import IndpTest_LKWeightGaussian
from Test.hsic_lkwlap import IndpTest_LKWeightLaplace
from Test.hsic_lkselect import IndpTest_LKSelect_GauLap
from Test.hsic_kselect import IndpTest_KSelect

device = torch.device('cuda:0')

In [4]:
def generate_ISA(n,d,sigma_normal,alpha):
    
    x = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    y = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    p = np.random.permutation(n)
    y_p = y[p]

    D = np.zeros([2,n])
    D[0,:] = x
    D[1,:] = y_p

    theta = np.pi/4*alpha
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))

    D_R = R@D
    X_mix = D_R[0,:].reshape(-1,1)
    Y_mix = D_R[1,:].reshape(-1,1)

    X_z = np.random.randn(n,d-1)
    Y_z = np.random.randn(n,d-1)

    X_con = np.concatenate((X_mix,X_z), axis=1)
    Y_con = np.concatenate((Y_mix,Y_z), axis=1)

    m_x = ortho_group.rvs(dim=d)
    m_y = ortho_group.rvs(dim=d)

    X = (m_x@X_con.T).T
    Y = (m_y@Y_con.T).T
    
    return X,Y

In [6]:
# ica experiments #

test_num = 100
# result_all_pvalue = []
result_all_correct = []

alpha_set = np.linspace(0.0, 1.0, num=10)
# alpha_set = [1.0]

confident_level = 0.05

for alpha in alpha_set:
    result_test_correct = np.zeros([8,test_num])
    
    for it in range(test_num):
        n = 128
        d = 4
        sigma_normal = 0.1
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
#         #test#
#         hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
#         results_all0 = hsic0.perform_test()
#         result_test_correct[0, it] = float(results_all0['h0_rejected'])

#         hsic1 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Laplace", null_gamma = True)
#         results_all1 = hsic1.perform_test()
#         result_test_correct[1, it] = float(results_all1['h0_rejected'])

#         hsic2 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
#         results_all2 = hsic2.perform_test(debug = -1)
#         result_test_correct[2, it] = float(results_all2['h0_rejected'])

#         hsic3 = IndpTest_LKLaplace(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
#         results_all3 = hsic3.perform_test(debug = -1)
#         result_test_correct[3, it] = float(results_all3['h0_rejected'])

#         hsic4 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
#         results_all4 = hsic4.perform_test(debug = -1)
#         result_test_correct[4, it] = float(results_all4['h0_rejected'])

#         hsic5 = IndpTest_LKWeightLaplace(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
#         results_all5 = hsic5.perform_test(debug = -1)
#         result_test_correct[5, it] = float(results_all5['h0_rejected'])
        
        hsic6 = IndpTest_KSelect(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all6 = hsic6.perform_test(debug = -1)
        result_test_correct[6, it] = float(results_all6['h0_rejected'])
        
        # hsic7 = IndpTest_LKSelect_GauLap(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        # results_all7 = hsic7.perform_test(debug = -1)
        # result_test_correct[7, it] = float(results_all7['h0_rejected'])

        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    
    # result_all_pvalue.append(result_test_pvalue)
    result_all_correct.append(result_test_correct)

# np.save(f"D:/Learning kernel/Res/rbf_Gsign_n{n}_d{d}_split{split_ratio}_confident_{confident_level}_p.npy",result_test_pvalue)
# np.save(f"D:/Learning kernel/Res/rbf_Gsign_n{n}_d{d}_split{split_ratio}_confident_{confident_level}_c.npy",result_test_correct)

0.0 0 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 1 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 2 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 3 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 4 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 5 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 6 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 7 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 8 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 9 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 10 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 11 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 12 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 13 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 14 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 15 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 16 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 17 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 18 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 19 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 20 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 21 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 22 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 23 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 24 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 25 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 26 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 27 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 28 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 29 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 30 [0. 0. 0. 0. 

In [7]:
rl = []
for i in range(10):
    r = np.sum(result_all_correct[i],1)/100
    print(r)
    rl.append(r)

[0. 0. 0. 0. 0. 0. 0. 0.]
[0.   0.   0.   0.   0.   0.   0.03 0.  ]
[0.   0.   0.   0.   0.   0.   0.02 0.  ]
[0.   0.   0.   0.   0.   0.   0.08 0.  ]
[0.   0.   0.   0.   0.   0.   0.13 0.  ]
[0.   0.   0.   0.   0.   0.   0.26 0.  ]
[0.   0.   0.   0.   0.   0.   0.35 0.  ]
[0.   0.   0.   0.   0.   0.   0.49 0.  ]
[0.   0.   0.   0.   0.   0.   0.51 0.  ]
[0.  0.  0.  0.  0.  0.  0.4 0. ]


In [11]:
# ica experiments #

test_num = 100
# result_all_pvalue = []
result_all_correct = []

alpha_set = np.linspace(0.0, 1.0, num=10)
# alpha_set = [1.0]

confident_level = 0.05

for alpha in alpha_set:
    result_test_correct = np.zeros([8,test_num])
    
    for it in range(test_num):
        n = 128
        d = 4
        sigma_normal = 0.1
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
        #test#
        hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
        results_all0 = hsic0.perform_test()
        result_test_correct[0, it] = float(results_all0['h0_rejected'])

        hsic1 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Laplace", null_gamma = True)
        results_all1 = hsic1.perform_test()
        result_test_correct[1, it] = float(results_all1['h0_rejected'])

        hsic2 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all2 = hsic2.perform_test(debug = -1)
        result_test_correct[2, it] = float(results_all2['h0_rejected'])

        hsic3 = IndpTest_LKLaplace(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all3 = hsic3.perform_test(debug = -1)
        result_test_correct[3, it] = float(results_all3['h0_rejected'])

        hsic4 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all4 = hsic4.perform_test(debug = -1)
        result_test_correct[4, it] = float(results_all4['h0_rejected'])

        hsic5 = IndpTest_LKWeightLaplace(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all5 = hsic5.perform_test(debug = -1)
        result_test_correct[5, it] = float(results_all5['h0_rejected'])
        
        hsic6 = IndpTest_KSelect(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all6 = hsic6.perform_test(debug = -1)
        result_test_correct[6, it] = float(results_all6['h0_rejected'])
        
        hsic7 = IndpTest_LKSelect_GauLap(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all7 = hsic7.perform_test(debug = -1)
        result_test_correct[7, it] = float(results_all7['h0_rejected'])

        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    
    # result_all_pvalue.append(result_test_pvalue)
    result_all_correct.append(result_test_correct)

# np.save(f"D:/Learning kernel/Res/rbf_Gsign_n{n}_d{d}_split{split_ratio}_confident_{confident_level}_p.npy",result_test_pvalue)
# np.save(f"D:/Learning kernel/Res/rbf_Gsign_n{n}_d{d}_split{split_ratio}_confident_{confident_level}_c.npy",result_test_correct)

0.0 0 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 1 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 2 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 3 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 4 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 5 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 6 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 7 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 8 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 9 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 10 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 11 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 12 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 13 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 14 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 15 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 16 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 17 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 18 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 19 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 20 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 21 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 22 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 23 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 24 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 25 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 26 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 27 [0. 0. 0. 0. 0. 0. 0. 0.]
0.0 28 [0.         0.03448276 0.         0.         0.         0.
 0.         0.      

In [15]:
rl = []
for i in range(10):
    r = np.sum(result_all_correct[i],1)/100
    print(r)
    rl.append(r)

[0.   0.04 0.02 0.01 0.   0.03 0.02 0.  ]
[0.02 0.01 0.01 0.   0.05 0.01 0.03 0.01]
[0.03 0.06 0.03 0.04 0.13 0.15 0.02 0.02]
[0.04 0.16 0.06 0.04 0.52 0.34 0.04 0.08]
[0.06 0.21 0.33 0.04 0.61 0.45 0.15 0.33]
[0.04 0.2  0.35 0.1  0.81 0.56 0.17 0.38]
[0.01 0.3  0.26 0.16 0.83 0.57 0.33 0.32]
[0.09 0.31 0.33 0.08 0.87 0.5  0.45 0.31]
[0.07 0.23 0.37 0.1  0.79 0.43 0.45 0.32]
[0.14 0.2  0.39 0.1  0.86 0.36 0.42 0.38]


In [18]:
# np.save(f"./202422.npy",np.array(rl))

In [17]:
np.array(rl)

array([[0.  , 0.04, 0.02, 0.01, 0.  , 0.03, 0.02, 0.  ],
       [0.02, 0.01, 0.01, 0.  , 0.05, 0.01, 0.03, 0.01],
       [0.03, 0.06, 0.03, 0.04, 0.13, 0.15, 0.02, 0.02],
       [0.04, 0.16, 0.06, 0.04, 0.52, 0.34, 0.04, 0.08],
       [0.06, 0.21, 0.33, 0.04, 0.61, 0.45, 0.15, 0.33],
       [0.04, 0.2 , 0.35, 0.1 , 0.81, 0.56, 0.17, 0.38],
       [0.01, 0.3 , 0.26, 0.16, 0.83, 0.57, 0.33, 0.32],
       [0.09, 0.31, 0.33, 0.08, 0.87, 0.5 , 0.45, 0.31],
       [0.07, 0.23, 0.37, 0.1 , 0.79, 0.43, 0.45, 0.32],
       [0.14, 0.2 , 0.39, 0.1 , 0.86, 0.36, 0.42, 0.38]])

In [19]:
# np.save(f"./202422al.npy",result_all_correct)

In [18]:
n = 128
d = 4
sigma_normal = 0.1
alpha = 1.0
X, Y = generate_ISA(n,d,sigma_normal,alpha)
X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
aa = dcor.independence.distance_covariance_test(X, Y, num_resamples=len(X))
aa

HypothesisTest(p_value=0.27906976744186046, statistic=7.443591681134642)

In [9]:
aa.p_value

0.06976744186046512

In [19]:
test_num = 100
# result_all_pvalue = []
result_all_correct = []

alpha_set = np.linspace(0.0, 1.0, num=10)
# alpha_set = [1.0]

confident_level = 0.05

p_all = []

for alpha in alpha_set:
    result_test_correct = np.zeros([1,test_num])
    p = []
    for it in range(test_num):
        n = 128
        d = 4
        sigma_normal = 0.1
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
        aa = dcor.independence.distance_covariance_test(X, Y, num_resamples=len(X))
        p.append(aa.p_value)
        result_test_correct[0, it] = float(aa.p_value<=confident_level)
        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    p_all.append(p)
    result_all_correct.append(result_test_correct)

0.0 0 [0.]
0.0 1 [0.]
0.0 2 [0.]
0.0 3 [0.]
0.0 4 [0.]
0.0 5 [0.]
0.0 6 [0.]
0.0 7 [0.]
0.0 8 [0.]
0.0 9 [0.]
0.0 10 [0.]
0.0 11 [0.]
0.0 12 [0.07692308]
0.0 13 [0.07142857]
0.0 14 [0.06666667]
0.0 15 [0.0625]
0.0 16 [0.05882353]
0.0 17 [0.05555556]
0.0 18 [0.05263158]
0.0 19 [0.05]
0.0 20 [0.04761905]
0.0 21 [0.04545455]
0.0 22 [0.04347826]
0.0 23 [0.04166667]
0.0 24 [0.04]
0.0 25 [0.03846154]
0.0 26 [0.03703704]
0.0 27 [0.03571429]
0.0 28 [0.06896552]
0.0 29 [0.06666667]
0.0 30 [0.06451613]
0.0 31 [0.0625]
0.0 32 [0.06060606]
0.0 33 [0.05882353]
0.0 34 [0.05714286]
0.0 35 [0.05555556]
0.0 36 [0.05405405]
0.0 37 [0.05263158]
0.0 38 [0.05128205]
0.0 39 [0.05]
0.0 40 [0.04878049]
0.0 41 [0.04761905]
0.0 42 [0.04651163]
0.0 43 [0.04545455]
0.0 44 [0.04444444]
0.0 45 [0.04347826]
0.0 46 [0.04255319]
0.0 47 [0.04166667]
0.0 48 [0.04081633]
0.0 49 [0.04]
0.0 50 [0.03921569]
0.0 51 [0.03846154]
0.0 52 [0.03773585]
0.0 53 [0.03703704]
0.0 54 [0.03636364]
0.0 55 [0.03571429]
0.0 56 [0.03508772

In [20]:
result_all_correct

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,

In [22]:
rl = []
for i in range(10):
    r = np.sum(result_all_correct[i],1)/100
    print(r)
    rl.append(r)

[0.05]
[0.06]
[0.06]
[0.12]
[0.13]
[0.12]
[0.12]
[0.08]
[0.03]
[0.06]


In [23]:
rl

[array([0.05]),
 array([0.06]),
 array([0.06]),
 array([0.12]),
 array([0.13]),
 array([0.12]),
 array([0.12]),
 array([0.08]),
 array([0.03]),
 array([0.06])]

In [24]:
# np.save(f"./dcor.npy",np.array(rl))

In [ ]:
np.save(f"./dcor.npy",np.array(rl))